## Market price prediction using Technical Indicators 

In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import pandas as pd
import matplotlib.pyplot as plt 
import statsmodels.formula.api as sm
from sklearn.model_selection import train_test_split
import numpy as np
from statsmodels.stats.stattools import durbin_watson
from statsmodels.regression.linear_model import OLS

In [2]:
dataset=pd.read_csv("../Datasets/BSE.csv",
                    usecols=[ 'open', 'high', 'low',  
        'volume', 'SMA', 'EMA','WMA', 'DEMA', 'TEMA', 'TRIMA', 'KAMA', 
        'MAMA-FAMA', 'MAMA-MAMA', 'T3','MACD-MACD', 'MACD-MACD_Hist',
        'MACD-MACD_Signal', 'MACDEXT-MACD','MACDEXT-MACD_Hist', 'MACDEXT-MACD_Signal',
        'SlowD', 'SlowK','STOCHF-FastD', 'STOCKF-FastK', 'RSI', 'STOCHRSI-FastD',
       'STOCHRSI-FastK', 'WILLR', 'ADX', 'ADXR', 'APO', 'PPO']
                   )
dataset.head()

,open,high,low,volume,SMA,EMA,WMA,DEMA,TEMA,TRIMA,...,STOCHF-FastD,STOCKF-FastK,RSI,STOCHRSI-FastD,STOCHRSI-FastK,WILLR,ADX,ADXR,APO,PPO
0,36304.4297,36389.2188,36082.9688,23300,35600.2137,35626.1063,35688.0792,35939.9651,36045.0672,35488.1156,...,89.8132,86.9458,68.7552,100.0000,100.000,-13.0542,18.9468,19.0408,535.9070,1.5310
1,35997.2891,36253.8516,35946.2383,40500,35484.3457,35475.3678,35538.9730,35708.2344,35755.7004,35453.6277,...,85.4471,93.8546,64.6462,80.2607,100.000,-6.1454,16.8818,18.0749,505.4852,1.4478
2,35635.5195,35822.1602,35605.3398,27500,35417.6309,35359.3853,35433.5806,35528.0176,35521.7972,35447.7939,...,70.7071,88.6392,58.9192,46.9273,100.000,-11.3608,15.0797,17.6229,496.6389,1.4246
3,35394.7695,35555.1602,35262.9688,19100,35338.6980,35298.0221,35379.6131,35442.7651,35411.2731,35429.2964,...,42.7894,73.8474,54.5052,13.5940,40.782,-26.6597,14.9778,18.2013,506.4294,1.4544
4,35118.0898,35397.2383,34896.0703,19200,35327.9699,35276.5227,35367.4677,35431.9313,35404.1068,35395.6194,...,22.2940,49.6347,48.8170,12.1071,0.000,-40.5319,16.4131,20.0469,492.9727,1.4155


In [3]:
dataset.shape

(2695, 32)

In [4]:
X=dataset.iloc[:,:].values
Y= pd.read_csv("../Datasets/BSE.csv",
               usecols=['close']).iloc[:,:].values
# Now that we have set our X and Y 

### The Durbin Watson autocorrelation testing 

Brief notes on Durbin watson test. **Source**: [Wikipedia article on Durbin Watson](https://en.wikipedia.org/wiki/Durbin%E2%80%93Watson_statistic)

\begin{equation}
\frac{\sum_{t=2}^T((e_t - e_{t-1})^2))}{\sum_{t=1}^Te_t^2}
\end{equation}

**Here** 
**T**- Number of observations where e_t is the residual associated with the observation at time t with respect to an OLS regression. 

Durbin watson measures similarity of data over time series t
The Hypotheses for the Durbin Watson test are:
H0 = no first order autocorrelation.
H1 = first order correlation exists.

The Durbin Watson test reports a test statistic, with a value from 0 to 4, where:

   1. 2- is no autocorrelation.
   2. 0 to <2 is positive autocorrelation (common in time series data).
   3. \>2 to 4 is negative autocorrelation (less common in time series data).


In [5]:
import math
def average(x):
    assert len(x) > 0
    return float(sum(x)) / len(x)
def pearson_def(x, y):
    assert len(x) == len(y)
    n = len(x)
    assert n > 0
    avg_x = average(x)
    avg_y = average(y)
    diffprod = 0
    xdiff2 = 0
    ydiff2 = 0
    for idx in range(n):
        xdiff = x[idx] - avg_x
        ydiff = y[idx] - avg_y
        diffprod += xdiff * ydiff
        xdiff2 += xdiff * xdiff
        ydiff2 += ydiff * ydiff
    return diffprod / math.sqrt(xdiff2 * ydiff2)

In [6]:
# Everything vs closing prices Pearson correlation test
a= dataset.columns 
closingpearson= list()
for i in range(0,32):
    print(a[i]+" vs closing ---> "+str(pearson_def(X[:,i], Y[:,0])))
    closingpearson.append(pearson_def(X[:,i], Y[:,0]))

open vs closing ---> 0.9995536498595958
high vs closing ---> 0.9997959539123246
low vs closing ---> 0.9998344415913696
volume vs closing ---> -0.21500308287111963
SMA vs closing ---> 0.9705842378194656
EMA vs closing ---> 0.9764887442717711
WMA vs closing ---> 0.9760276451544717
DEMA vs closing ---> 0.9825054983212432
TEMA vs closing ---> 0.9864254649764018
TRIMA vs closing ---> 0.9656792547768334
KAMA vs closing ---> 0.9430966863164165
MAMA-FAMA vs closing ---> 0.9051751767593803
MAMA-MAMA vs closing ---> 0.9436593421446502
T3 vs closing ---> 0.9644817216437319
MACD-MACD vs closing ---> 0.26331058709187316
MACD-MACD_Hist vs closing ---> 0.160244285781585
MACD-MACD_Signal vs closing ---> 0.2208518232179358
MACDEXT-MACD vs closing ---> 0.15087348703680958
MACDEXT-MACD_Hist vs closing ---> 0.0619415891661173
MACDEXT-MACD_Signal vs closing ---> 0.1257721100594846
SlowD vs closing ---> 0.07972685241512724
SlowK vs closing ---> 0.08438360809596918
STOCHF-FastD vs closing ---> 0.084383608095

In [7]:
a=dataset.columns 
for i in range(0,32):
    for j in range(i+1,32):
        print(a[i]+" vs "+a[j]+"---->"+str(pearson_def(X[:,i],X[:,j])))

open vs high---->0.9998489236771398
open vs low---->0.9997425057790835
open vs volume---->-0.21403185593022903
open vs SMA---->0.971055084179541
open vs EMA---->0.9769924947688328
open vs WMA---->0.9765025463620001
open vs DEMA---->0.9829687900952694
open vs TEMA---->0.9868842027656654
open vs TRIMA---->0.9661418803161927
open vs KAMA---->0.9434855845171758
open vs MAMA-FAMA---->0.9052395600208177
open vs MAMA-MAMA---->0.9440987544120555
open vs T3---->0.9649901411663201
open vs MACD-MACD---->0.26293384621538446
open vs MACD-MACD_Hist---->0.15966884722986052
open vs MACD-MACD_Signal---->0.2207364940051338
open vs MACDEXT-MACD---->0.15008545994941846
open vs MACDEXT-MACD_Hist---->0.06100199787112238
open vs MACDEXT-MACD_Signal---->0.12554237124198833
open vs SlowD---->0.0774949819472908
open vs SlowK---->0.07723599414287173
open vs STOCHF-FastD---->0.07723599414287173
open vs STOCKF-FastK---->0.07537922393942449
open vs RSI---->0.27122858687949486
open vs STOCHRSI-FastD---->0.0452443999

DEMA vs STOCHRSI-FastK---->0.001710727146298063
DEMA vs WILLR---->0.05562309425426213
DEMA vs ADX---->0.1329785062017027
DEMA vs ADXR---->0.15066461756562016
DEMA vs APO---->0.17039213013241641
DEMA vs PPO---->0.14533522258970605
TEMA vs TRIMA---->0.9880284390302686
TEMA vs KAMA---->0.9631751540159745
TEMA vs MAMA-FAMA---->0.921397651525483
TEMA vs MAMA-MAMA---->0.9610198257788937
TEMA vs T3---->0.9853285514479548
TEMA vs MACD-MACD---->0.26650235104454734
TEMA vs MACD-MACD_Hist---->0.12358673711220998
TEMA vs MACD-MACD_Signal---->0.24590436527421944
TEMA vs MACDEXT-MACD---->0.16737103261622707
TEMA vs MACDEXT-MACD_Hist---->0.07322237685640227
TEMA vs MACDEXT-MACD_Signal---->0.13639926383638404
TEMA vs SlowD---->0.07111388489524752
TEMA vs SlowK---->0.05483392916370099
TEMA vs STOCHF-FastD---->0.05483392916370099
TEMA vs STOCKF-FastK---->0.03154263713961058
TEMA vs RSI---->0.2356271061456948
TEMA vs STOCHRSI-FastD---->0.026270622785866535
TEMA vs STOCHRSI-FastK---->0.011675128975488289


MACDEXT-MACD vs STOCHRSI-FastK---->-0.05897976381434836
MACDEXT-MACD vs WILLR---->0.06803276585108535
MACDEXT-MACD vs ADX---->-0.14681903121621975
MACDEXT-MACD vs ADXR---->-0.15841157249948304
MACDEXT-MACD vs APO---->1.0
MACDEXT-MACD vs PPO---->0.9464961793403815
MACDEXT-MACD_Hist vs MACDEXT-MACD_Signal---->-0.16599402530167875
MACDEXT-MACD_Hist vs SlowD---->0.18385644065537374
MACDEXT-MACD_Hist vs SlowK---->0.14001031910468964
MACDEXT-MACD_Hist vs STOCHF-FastD---->0.14001031910468964
MACDEXT-MACD_Hist vs STOCKF-FastK---->0.08228652956992336
MACDEXT-MACD_Hist vs RSI---->0.25543514505320347
MACDEXT-MACD_Hist vs STOCHRSI-FastD---->0.10454447109325452
MACDEXT-MACD_Hist vs STOCHRSI-FastK---->0.05615244105477184
MACDEXT-MACD_Hist vs WILLR---->0.17370081907228663
MACDEXT-MACD_Hist vs ADX---->-0.10421610749837446
MACDEXT-MACD_Hist vs ADXR---->-0.038403154895590944
MACDEXT-MACD_Hist vs APO---->0.47161518790982
MACDEXT-MACD_Hist vs PPO---->0.43838650700863513
MACDEXT-MACD_Signal vs SlowD---->-0

In [8]:
pearsonmatrix= np.empty([32, 32])
for i in range(0,32):
    for j in range(0,32):
        pearsonmatrix[i][j]=pearson_def(X[:,i],X[:,j])

In [9]:
pearsonmatrix

array([[ 1.        ,  0.99984892,  0.99974251, ...,  0.14700858,
         0.15008546,  0.12676591],
       [ 0.99984892,  1.        ,  0.99974922, ...,  0.14793185,
         0.14964603,  0.12618845],
       [ 0.99974251,  0.99974922,  1.        , ...,  0.14618918,
         0.15206395,  0.12859993],
       ...,
       [ 0.14700858,  0.14793185,  0.14618918, ...,  1.        ,
        -0.15841157, -0.17458039],
       [ 0.15008546,  0.14964603,  0.15206395, ..., -0.15841157,
         1.        ,  0.94649618],
       [ 0.12676591,  0.12618845,  0.12859993, ..., -0.17458039,
         0.94649618,  1.        ]])

In [10]:
dw_list=[]
for i in range(0,32):
    dw_list.append( durbin_watson(X[:,i]))
    print(durbin_watson(X[:,i]))

0.010119380012150764
0.010109124626263693
0.010102929818260858
0.7771341702293111
0.0001230674163769384
0.0002028673256601857
0.00021008625977222902
0.0006854264593000593
0.0013307933890742002
8.701298677537586e-05
0.000909741947003976
2.592340787793562e-07
1.25052307994687e-06
3.952597494713776e-05
0.14620311093339552
0.4539676509953825
0.020315106316949997
0.0932526580435345
0.20537940246573191
0.023610868707829493
0.0334771983557205
0.05517669083691006
0.05517669083691006
0.20428052171459066
0.019146679187900244
0.0972473477584073
0.3762431219614248
0.15796486097138213
0.003562790710851028
0.0017349967790174195
0.0932526580435345
0.08655493662417195


In [11]:
# All OLS regressions to Output and regression summaries 
i=0
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

open
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.230e+07
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:07   Log-Likelihood:                -18268.
No. Observations:                2695   AIC:                         3.654e+04
Df Residuals:                    2694   BIC:                         3.654e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.0011      0.000   5683.411    

In [12]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

high
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 6.590e+07
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:07   Log-Likelihood:                -17321.
No. Observations:                2695   AIC:                         3.464e+04
Df Residuals:                    2694   BIC:                         3.465e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.0062      0.000   8118.184    

In [13]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

low
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.735e+07
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:07   Log-Likelihood:                -17073.
No. Observations:                2695   AIC:                         3.415e+04
Df Residuals:                    2694   BIC:                         3.415e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9942      0.000   8794.656     

In [14]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

volume
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.348
Method:                 Least Squares   F-statistic:                     1439.
Date:                Tue, 15 Jan 2019   Prob (F-statistic):          1.10e-252
Time:                        00:24:07   Log-Likelihood:                -30590.
No. Observations:                2695   AIC:                         6.118e+04
Df Residuals:                    2694   BIC:                         6.119e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.6464      0.017     37.935  

In [15]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

SMA
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                 4.952e+05
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:08   Log-Likelihood:                -23880.
No. Observations:                2695   AIC:                         4.776e+04
Df Residuals:                    2694   BIC:                         4.777e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9945      0.001    703.724     

In [16]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

EMA
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 6.180e+05
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:08   Log-Likelihood:                -23583.
No. Observations:                2695   AIC:                         4.717e+04
Df Residuals:                    2694   BIC:                         4.717e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9950      0.001    786.115     

In [17]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

WMA
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 6.065e+05
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:08   Log-Likelihood:                -23610.
No. Observations:                2695   AIC:                         4.722e+04
Df Residuals:                    2694   BIC:                         4.723e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9956      0.001    778.784     

In [18]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

DEMA
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 8.293e+05
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:08   Log-Likelihood:                -23193.
No. Observations:                2695   AIC:                         4.639e+04
Df Residuals:                    2694   BIC:                         4.639e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9975      0.001    910.639    

In [19]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

TEMA
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                 1.067e+06
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:08   Log-Likelihood:                -22855.
No. Observations:                2695   AIC:                         4.571e+04
Df Residuals:                    2694   BIC:                         4.572e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9982      0.001   1033.083    

In [20]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

TRIMA
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 4.249e+05
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:08   Log-Likelihood:                -24085.
No. Observations:                2695   AIC:                         4.817e+04
Df Residuals:                    2694   BIC:                         4.818e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9942      0.002    651.823   

In [21]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

KAMA
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                 2.338e+05
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:08   Log-Likelihood:                -24829.
No. Observations:                2695   AIC:                         4.966e+04
Df Residuals:                    2694   BIC:                         4.967e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9718      0.002    483.492    

In [22]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

MAMA-FAMA
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                 1.537e+05
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:08   Log-Likelihood:                -25139.
No. Observations:                2695   AIC:                         5.028e+04
Df Residuals:                    2694   BIC:                         5.029e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.8840      0.002    392.05

In [23]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

MAMA-MAMA
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 2.571e+05
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:09   Log-Likelihood:                -24658.
No. Observations:                2695   AIC:                         4.932e+04
Df Residuals:                    2694   BIC:                         4.932e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9566      0.002    507.05

In [24]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

T3
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                 4.106e+05
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:09   Log-Likelihood:                -24131.
No. Observations:                2695   AIC:                         4.826e+04
Df Residuals:                    2694   BIC:                         4.827e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.9942      0.002    640.783      

In [25]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

MACD-MACD
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     90.52
Date:                Tue, 15 Jan 2019   Prob (F-statistic):           3.91e-21
Time:                        00:24:09   Log-Likelihood:                -19835.
No. Observations:                2695   AIC:                         3.967e+04
Df Residuals:                    2694   BIC:                         3.968e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0030      0.000      9.51

In [26]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

MACD-MACD_Hist
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     6.215
Date:                Tue, 15 Jan 2019   Prob (F-statistic):             0.0127
Time:                        00:24:09   Log-Likelihood:                -17919.
No. Observations:                2695   AIC:                         3.584e+04
Df Residuals:                    2694   BIC:                         3.585e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0004      0.000     

In [27]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

MACD-MACD_Signal
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     97.33
Date:                Tue, 15 Jan 2019   Prob (F-statistic):           1.41e-22
Time:                        00:24:09   Log-Likelihood:                -19366.
No. Observations:                2695   AIC:                         3.873e+04
Df Residuals:                    2694   BIC:                         3.874e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0026      0.000   

In [28]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

MACDEXT-MACD
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     31.87
Date:                Tue, 15 Jan 2019   Prob (F-statistic):           1.82e-08
Time:                        00:24:09   Log-Likelihood:                -21094.
No. Observations:                2695   AIC:                         4.219e+04
Df Residuals:                    2694   BIC:                         4.220e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0028      0.001      5

In [29]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

MACDEXT-MACD_Hist
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.062
Date:                Tue, 15 Jan 2019   Prob (F-statistic):              0.303
Time:                        00:24:09   Log-Likelihood:                -19816.
No. Observations:                2695   AIC:                         3.963e+04
Df Residuals:                    2694   BIC:                         3.964e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0003      0.000  

In [30]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

MACDEXT-MACD_Signal
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     31.28
Date:                Tue, 15 Jan 2019   Prob (F-statistic):           2.46e-08
Time:                        00:24:10   Log-Likelihood:                -20794.
No. Observations:                2695   AIC:                         4.159e+04
Df Residuals:                    2694   BIC:                         4.160e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0025      0.000

In [31]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

SlowD
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.761
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     8564.
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:10   Log-Likelihood:                -12934.
No. Observations:                2695   AIC:                         2.587e+04
Df Residuals:                    2694   BIC:                         2.588e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0023   2.43e-05     92.540   

In [32]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

SlowK
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.740
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                     7656.
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:10   Log-Likelihood:                -13090.
No. Observations:                2695   AIC:                         2.618e+04
Df Residuals:                    2694   BIC:                         2.619e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0023   2.58e-05     87.498   

In [33]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

STOCHF-FastD
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.740
Model:                            OLS   Adj. R-squared:                  0.740
Method:                 Least Squares   F-statistic:                     7656.
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:10   Log-Likelihood:                -13090.
No. Observations:                2695   AIC:                         2.618e+04
Df Residuals:                    2694   BIC:                         2.619e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0023   2.58e-05     87

In [34]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

STOCKF-FastK
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.683
Model:                            OLS   Adj. R-squared:                  0.683
Method:                 Least Squares   F-statistic:                     5817.
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:10   Log-Likelihood:                -13472.
No. Observations:                2695   AIC:                         2.695e+04
Df Residuals:                    2694   BIC:                         2.695e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0023   2.97e-05     76

In [35]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

RSI
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.893
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                 2.239e+04
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:10   Log-Likelihood:                -11563.
No. Observations:                2695   AIC:                         2.313e+04
Df Residuals:                    2694   BIC:                         2.313e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0022   1.46e-05    149.632     

In [36]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

STOCHRSI-FastD
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     5240.
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:10   Log-Likelihood:                -13500.
No. Observations:                2695   AIC:                         2.700e+04
Df Residuals:                    2694   BIC:                         2.701e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0022      3e-05     

In [37]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

STOCHRSI-FastK
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.566
Method:                 Least Squares   F-statistic:                     3520.
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:10   Log-Likelihood:                -14048.
No. Observations:                2695   AIC:                         2.810e+04
Df Residuals:                    2694   BIC:                         2.810e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0022   3.68e-05     

In [38]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

WILLR
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.529
Method:                 Least Squares   F-statistic:                     3033.
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:11   Log-Likelihood:                -13555.
No. Observations:                2695   AIC:                         2.711e+04
Df Residuals:                    2694   BIC:                         2.712e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.0017   3.06e-05    -55.069   

In [39]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

ADX
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                 1.008e+04
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:11   Log-Likelihood:                -10637.
No. Observations:                2695   AIC:                         2.128e+04
Df Residuals:                    2694   BIC:                         2.128e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0010   1.04e-05    100.407     

In [40]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

ADXR
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.813
Model:                            OLS   Adj. R-squared:                  0.813
Method:                 Least Squares   F-statistic:                 1.172e+04
Date:                Tue, 15 Jan 2019   Prob (F-statistic):               0.00
Time:                        00:24:11   Log-Likelihood:                -10438.
No. Observations:                2695   AIC:                         2.088e+04
Df Residuals:                    2694   BIC:                         2.088e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0010   9.64e-06    108.247    

In [41]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

APO
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     31.87
Date:                Tue, 15 Jan 2019   Prob (F-statistic):           1.82e-08
Time:                        00:24:11   Log-Likelihood:                -21094.
No. Observations:                2695   AIC:                         4.219e+04
Df Residuals:                    2694   BIC:                         4.220e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0028      0.001      5.645     

In [42]:
print(a[i])
print(OLS(X[:,i],Y[:,0]).fit().summary())
i=i+1

PPO
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     19.28
Date:                Tue, 15 Jan 2019   Prob (F-statistic):           1.17e-05
Time:                        00:24:11   Log-Likelihood:                -6917.9
No. Observations:                2695   AIC:                         1.384e+04
Df Residuals:                    2694   BIC:                         1.384e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1          1.146e-05   2.61e-06      4.390     

In [43]:
dataset

,open,high,low,volume,SMA,EMA,WMA,DEMA,TEMA,TRIMA,...,STOCHF-FastD,STOCKF-FastK,RSI,STOCHRSI-FastD,STOCHRSI-FastK,WILLR,ADX,ADXR,APO,PPO
0,36304.4297,36389.2188,36082.9688,23300,35600.2137,35626.1063,35688.0792,35939.9651,36045.0672,35488.1156,...,89.8132,86.9458,68.7552,100.0000,100.0000,-13.0542,18.9468,19.0408,535.9070,1.5310
1,35997.2891,36253.8516,35946.2383,40500,35484.3457,35475.3678,35538.9730,35708.2344,35755.7004,35453.6277,...,85.4471,93.8546,64.6462,80.2607,100.0000,-6.1454,16.8818,18.0749,505.4852,1.4478
2,35635.5195,35822.1602,35605.3398,27500,35417.6309,35359.3853,35433.5806,35528.0176,35521.7972,35447.7939,...,70.7071,88.6392,58.9192,46.9273,100.0000,-11.3608,15.0797,17.6229,496.6389,1.4246
3,35394.7695,35555.1602,35262.9688,19100,35338.6980,35298.0221,35379.6131,35442.7651,35411.2731,35429.2964,...,42.7894,73.8474,54.5052,13.5940,40.7820,-26.6597,14.9778,18.2013,506.4294,1.4544
4,35118.0898,35397.2383,34896.0703,19200,35327.9699,35276.5227,35367.4677,35431.9313,35404.1068,35395.6194,...,22.2940,49.6347,48.8170,12.1071,0.0000,-40.5319,16.4131,20.0469,492.9727,1.4155
5,35282.3281,35364.5000,34937.9805,13800,35341.9738,35311.7300,35408.1739,35536.8812,35572.6160,35348.8284,...,25.2872,4.8861,52.3112,45.4404,0.0000,-73.0671,17.4114,21.5697,493.8606,1.4182
6,35492.6211,35494.2500,35112.4883,13900,35343.9289,35318.2638,35419.3740,35599.9823,35700.2255,35297.8880,...,55.7142,12.3613,57.0139,78.7738,36.3213,-53.9867,18.7642,22.6566,492.1606,1.4138
7,35730.7695,35731.6719,35416.1797,15700,35302.2906,35279.5177,35384.7685,35585.0943,35731.4717,35243.0368,...,81.5432,58.6143,62.7648,97.8960,100.0000,-30.0286,20.4051,23.3954,475.9520,1.3691
8,35647.6211,35818.8281,35511.1016,15600,35241.0758,35179.2395,35295.7332,35452.4438,35598.9773,35202.6827,...,88.8773,96.1671,61.5459,80.9180,100.0000,-3.8329,20.1480,23.3665,450.0537,1.2970
9,35398.6992,35545.8516,35324.3711,21100,35150.7086,35075.1547,35205.3855,35304.9863,35440.7101,35173.6965,...,79.8613,89.8482,57.8264,75.9188,93.6881,-9.8980,19.1348,23.0973,354.0765,1.0217
